In [1]:
import pandas as pd
import os
import numpy as np
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn.model_selection import train_test_split

pd.set_option("max_columns", 30)

In [31]:
t = pd.read_csv("../Data/train_complete.csv")['time'].apply(lambda x: x[:10])
pd.to_timedelta(pd.to_datetime(t), unit = 'seconds').astype('timedelta64[D]')[0]

c:\users\dylan\anaconda3\envs\kitties\lib\site-packages\ipykernel_launcher.py:2: FutureWarning: Passing datetime64-dtype data to TimedeltaIndex is deprecated, will raise a TypeError in a future version
  


18084.0

In [35]:
data = pd.read_csv("../Crawler/Latest_kitties_2019-07-07T20-08-25.csv")
# data['time'] = pd.to_timedelta(pd.to_datetime(pd.read_csv("../Data/train_complete.csv")['time'].apply(lambda x: x[:10])), unit = 'seconds').astype('timedelta64[D]')[0]
data['time'] = 18081.0
data['prestige_time_limit'] = data['prestige_time_limit'].apply(lambda x: np.nan if pd.isnull(x) else x[:10])
data['prestige_time_limit'] = pd.to_timedelta(pd.to_datetime(data['prestige_time_limit']), unit = 'seconds').astype('timedelta64[D]')

trends = pd.read_csv("../Data/trends.csv")

ETH = pd.read_csv("../Data/ETH_price.csv")
ETH = ETH[["date", "close"]]

data['trends'] = trends['cryptokitties']
data['close'] = ETH['close']

data['USD'] = data['USD'] * ETH['close'][0]

data = data.fillna('0')

data.head()

c:\users\dylan\anaconda3\envs\kitties\lib\site-packages\ipykernel_launcher.py:5: FutureWarning: Passing datetime64-dtype data to TimedeltaIndex is deprecated, will raise a TypeError in a future version
  """


,time,ID,USD,generation,color,is_fancy,is_exclusive,fancy_type,is_prestige,prestige_type,prestige_ranking,fancy_ranking,prestige_time_limit,enhanced_cattributes_eyes,enhanced_cattributes_pattern,enhanced_cattributes_body,enhanced_cattributes_mouth,enhanced_cattributes_colorprimary,enhanced_cattributes_colorsecondary,enhanced_cattributes_colortertiary,enhanced_cattributes_coloreyes,status_cooldown_index,status_is_ready,status_is_gestating,purrs_count,purrs_is_purred,watchlist_count,watchlist_is_watchlisted,trends,close
0,18081.0,1532879,23.004,9,forgetmenot,False,False,0,True,timbers,141,0,18026,beard,amur,mekong,swarley,cinderella,royalpurple,icy,forgetmenot,4,True,False,2,False,0,False,0.54453,287.55
1,18081.0,1460942,23.004,17,sapphire,False,False,0,False,0,0,0,0,confuzzled,chantilly,wonky,ganado,hintomint,coffee,sully,sapphire,8,True,False,0,False,0,False,0.877299,288
2,18081.0,1549546,23.004,5,mintgreen,False,False,0,False,0,0,0,0,beard,ragdoll,asif,thunderstruck,salty,orangesoda,wolfgrey,cobalt,4,True,False,0,False,0,False,0.514278,284.52
3,18081.0,1513584,23.004,4,dahlia,False,False,0,False,0,0,0,0,beard,wonky,savannah,tiger,salty,ducky,cottoncandy,lemonade,2,True,False,1,False,0,False,0.54453,303.1
4,18081.0,1132878,23.004,2,sapphire,False,False,0,False,0,0,0,0,sphynx,soserious,slyboots,rascal,bananacream,turtleback,patrickstarfish,sapphire,1,True,False,1,False,0,False,0.423523,291.6


In [5]:
from collections import defaultdict
d = defaultdict(LabelEncoder)

In [6]:
dirs = [x.split(".")[0] for x in os.listdir("Saved_encoder/")]
dirs

['color',
 'enhanced_cattributes_body',
 'enhanced_cattributes_coloreyes',
 'enhanced_cattributes_colorprimary',
 'enhanced_cattributes_colorsecondary',
 'enhanced_cattributes_colortertiary',
 'enhanced_cattributes_eyes',
 'enhanced_cattributes_mouth',
 'enhanced_cattributes_pattern',
 'fancy_type',
 'prestige_type']

## IMPORTING LABLEENCODERS

In [7]:
for e in dirs:
    d[e].classes_ = np.load(f'Saved_encoder/{e}.npy', allow_pickle = True)
    

In [8]:
cols = ['color', 'fancy_type', 'prestige_type', 'enhanced_cattributes_eyes', 'enhanced_cattributes_pattern', 'enhanced_cattributes_body', 'enhanced_cattributes_mouth', 'enhanced_cattributes_colorprimary', 'enhanced_cattributes_colorsecondary', 'enhanced_cattributes_colortertiary', 'enhanced_cattributes_coloreyes']

## ACCOUNTING FOR UNSEEN VARIABLES!!

In [9]:
import bisect

for c in cols:
    data[c] = data[c].map(lambda s: 'other' if s not in d[c].classes_ else s)
    
    le_classes = d[c].classes_.tolist()
    bisect.insort_left(le_classes, 'other')
    d[c].classes_ = le_classes
    

In [10]:
data[cols] = data[cols].apply(lambda x: d[x.name].transform(x))

In [11]:
data.head(10)

,time,ID,USD,generation,color,is_fancy,is_exclusive,fancy_type,is_prestige,prestige_type,prestige_ranking,fancy_ranking,prestige_time_limit,enhanced_cattributes_eyes,enhanced_cattributes_pattern,enhanced_cattributes_body,enhanced_cattributes_mouth,enhanced_cattributes_colorprimary,enhanced_cattributes_colorsecondary,enhanced_cattributes_colortertiary,enhanced_cattributes_coloreyes,status_cooldown_index,status_is_ready,status_is_gestating,purrs_count,purrs_is_purred,watchlist_count,watchlist_is_watchlisted,trends,close
0,18081.0,1532879,23.004,9,11,False,False,0,True,28,141,0,18026,11,4,109,183,37,140,67,52,4,True,False,2,False,0,False,0.54453,287.55
1,18081.0,1460942,23.004,17,24,False,False,0,False,0,0,0,0,29,29,198,74,95,33,129,117,8,True,False,0,False,0,False,0.877299,288
2,18081.0,1549546,23.004,5,16,False,False,0,False,0,0,0,0,11,126,8,188,165,119,147,28,4,True,False,0,False,0,False,0.514278,284.52
3,18081.0,1513584,23.004,4,7,False,False,0,False,0,0,0,0,11,185,154,189,165,52,27,74,2,True,False,1,False,0,False,0.54453,303.1
4,18081.0,1132878,23.004,2,24,False,False,0,False,0,0,0,0,123,153,164,151,15,173,100,117,1,True,False,1,False,0,False,0.423523,291.6
5,18081.0,1502777,23.004,2,18,False,False,0,False,0,0,0,0,90,35,139,35,79,77,8,93,1,True,False,0,False,0,False,0,293.64
6,18081.0,1355806,23.004,2,24,False,False,0,False,0,0,0,0,131,172,77,105,42,33,58,117,1,True,False,1,False,0,False,0.36302,290.7
7,18081.0,1540985,23.004,4,11,False,False,0,False,0,0,0,0,34,167,196,207,139,180,67,52,2,True,False,0,False,0,False,0.514278,320.06
8,18081.0,1116582,23.004,3,7,False,False,0,False,0,0,0,0,29,164,157,105,37,142,91,36,1,True,False,1,False,0,False,0.695788,311.23
9,18081.0,1313291,23.004,6,27,False,False,0,True,24,663,0,17904,29,126,55,183,160,113,80,67,3,True,False,0,False,0,False,1.42183,294.27


In [12]:
X = data.drop('USD', axis = 1)
X.head(5)

,time,ID,generation,color,is_fancy,is_exclusive,fancy_type,is_prestige,prestige_type,prestige_ranking,fancy_ranking,prestige_time_limit,enhanced_cattributes_eyes,enhanced_cattributes_pattern,enhanced_cattributes_body,enhanced_cattributes_mouth,enhanced_cattributes_colorprimary,enhanced_cattributes_colorsecondary,enhanced_cattributes_colortertiary,enhanced_cattributes_coloreyes,status_cooldown_index,status_is_ready,status_is_gestating,purrs_count,purrs_is_purred,watchlist_count,watchlist_is_watchlisted,trends,close
0,18081.0,1532879,9,11,False,False,0,True,28,141,0,18026,11,4,109,183,37,140,67,52,4,True,False,2,False,0,False,0.54453,287.55
1,18081.0,1460942,17,24,False,False,0,False,0,0,0,0,29,29,198,74,95,33,129,117,8,True,False,0,False,0,False,0.877299,288
2,18081.0,1549546,5,16,False,False,0,False,0,0,0,0,11,126,8,188,165,119,147,28,4,True,False,0,False,0,False,0.514278,284.52
3,18081.0,1513584,4,7,False,False,0,False,0,0,0,0,11,185,154,189,165,52,27,74,2,True,False,1,False,0,False,0.54453,303.1
4,18081.0,1132878,2,24,False,False,0,False,0,0,0,0,123,153,164,151,15,173,100,117,1,True,False,1,False,0,False,0.423523,291.6


In [13]:
model_name = os.listdir("Saved_models/")[-1]

In [14]:
import pickle
loaded_model = pickle.load(open(f"Saved_models/{model_name}", 'rb'))
res = loaded_model.predict(X)
print(res)

[Parallel(n_jobs=6)]: Using backend ThreadingBackend with 6 concurrent workers.
[Parallel(n_jobs=6)]: Done  20 tasks      | elapsed:    0.0s


[10.3712      2.11621667  4.77598905 ...  5.4283      2.59313667
 16.5598    ]


[Parallel(n_jobs=6)]: Done 100 out of 100 | elapsed:    0.2s finished


In [37]:
data['predictions'] = res

In [38]:
data.head()

,time,ID,USD,generation,color,is_fancy,is_exclusive,fancy_type,is_prestige,prestige_type,prestige_ranking,fancy_ranking,prestige_time_limit,enhanced_cattributes_eyes,enhanced_cattributes_pattern,...,enhanced_cattributes_mouth,enhanced_cattributes_colorprimary,enhanced_cattributes_colorsecondary,enhanced_cattributes_colortertiary,enhanced_cattributes_coloreyes,status_cooldown_index,status_is_ready,status_is_gestating,purrs_count,purrs_is_purred,watchlist_count,watchlist_is_watchlisted,trends,close,predictions
0,18081.0,1532879,23.004,9,forgetmenot,False,False,0,True,timbers,141,0,18026,beard,amur,...,swarley,cinderella,royalpurple,icy,forgetmenot,4,True,False,2,False,0,False,0.54453,287.55,10.371200
1,18081.0,1460942,23.004,17,sapphire,False,False,0,False,0,0,0,0,confuzzled,chantilly,...,ganado,hintomint,coffee,sully,sapphire,8,True,False,0,False,0,False,0.877299,288,2.116217
2,18081.0,1549546,23.004,5,mintgreen,False,False,0,False,0,0,0,0,beard,ragdoll,...,thunderstruck,salty,orangesoda,wolfgrey,cobalt,4,True,False,0,False,0,False,0.514278,284.52,4.775989
3,18081.0,1513584,23.004,4,dahlia,False,False,0,False,0,0,0,0,beard,wonky,...,tiger,salty,ducky,cottoncandy,lemonade,2,True,False,1,False,0,False,0.54453,303.1,4.589833
4,18081.0,1132878,23.004,2,sapphire,False,False,0,False,0,0,0,0,sphynx,soserious,...,rascal,bananacream,turtleback,patrickstarfish,sapphire,1,True,False,1,False,0,False,0.423523,291.6,0.431173


In [15]:
results = dict()

In [16]:
results["predictions"] =  res
results["sale_price"] = data['USD']
results["ID"] = data['ID']

In [18]:
k = 0
best = []
print("")
for i in range(len(res)):
    if results["predictions"][i] > results["sale_price"][i]*15:
        print(round(results["predictions"][i],2), round(results["sale_price"][i],2), results["ID"][i])
        k+=1
        best.append([round(results["predictions"][i],2), round(results["sale_price"][i],2), results["ID"][i]])
print(k)


405.21 23.0 910535
978.66 23.94 845684
473.72 27.32 631055
676.59 34.6 597909
1096.71 57.51 1105494
1601.69 71.89 1130017
1646.99 79.1 1130047
1315.39 83.39 969183
8


In [151]:
from IPython.core.display import display, HTML


display(HTML(' <a href="google.com"><h5>Hello, world!</h5>'))

In [148]:
best

[[381.61, 25.6, 560270],
 [378.29, 25.6, 500361],
 [368.35, 25.6, 467130],
 [358.96, 25.6, 468434],
 [407.85, 25.6, 467737],
 [414.56, 26.76, 845684]]